In [1]:
import numpy as np
import pickle
from pathlib import Path
from scipy.spatial.distance import pdist
from itertools import combinations,chain
import pandas as pd

In [2]:
calval_dir = Path.cwd()/'calval'
calval_location = 'central_valley'
work_dir = calval_dir/calval_location

In [3]:
with open(work_dir/'approach1.pkl','rb') as f:
    dist, rel_measure, ifgs_date = pickle.load(f)

In [4]:
n_ifgs = len(dist)

In [5]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [6]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [7]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

In [8]:
ratio = n_pass/n_all
thresthod = 0.683 
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

In [9]:
columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

In [10]:
index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

In [11]:
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns,index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns,index=index)

Number of data points in each bin:

In [12]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20170108-20170120,42,92,102,121,120,119,136,121,106,102,959
20170114-20170126,43,92,103,124,118,125,138,129,113,107,985
20170120-20170201,39,91,101,120,114,119,125,121,109,98,939
20170225-20170309,46,88,100,124,118,122,139,141,120,109,998
20170321-20170402,42,92,102,125,122,127,142,143,119,110,1014
20170414-20170426,43,93,103,127,124,129,144,147,121,113,1031
20170508-20170520,49,101,107,135,127,134,151,151,125,113,1080
20170601-20170613,47,93,103,128,123,129,145,147,122,112,1037
20170625-20170707,47,93,103,128,123,129,145,147,122,112,1037
20170719-20170731,42,86,98,118,111,110,132,128,106,90,931


Number of data points that below the curve:

In [13]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20170108-20170120,26,58,87,107,110,104,126,112,97,93,827
20170114-20170126,41,81,92,118,117,121,137,129,109,101,945
20170120-20170201,30,63,82,94,101,106,107,105,94,88,782
20170225-20170309,38,72,83,93,94,106,118,116,95,70,815
20170321-20170402,35,53,65,67,67,77,67,67,62,56,560
20170414-20170426,39,73,80,100,84,83,80,96,83,75,718
20170508-20170520,42,83,90,119,115,128,143,144,115,100,979
20170601-20170613,44,79,95,122,119,126,144,147,118,106,994
20170625-20170707,43,69,80,98,93,93,119,113,84,70,792
20170719-20170731,38,82,81,90,79,82,102,100,79,67,733


Ratio of pass:

In [14]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20170108-20170120,0.619048,0.630435,0.852941,0.884298,0.916667,0.873950,0.926471,0.925620,0.915094,0.911765,0.862357
20170114-20170126,0.953488,0.880435,0.893204,0.951613,0.991525,0.968000,0.992754,1.000000,0.964602,0.943925,0.959391
20170120-20170201,0.769231,0.692308,0.811881,0.783333,0.885965,0.890756,0.856000,0.867769,0.862385,0.897959,0.832801
20170225-20170309,0.826087,0.818182,0.830000,0.750000,0.796610,0.868852,0.848921,0.822695,0.791667,0.642202,0.816633
20170321-20170402,0.833333,0.576087,0.637255,0.536000,0.549180,0.606299,0.471831,0.468531,0.521008,0.509091,0.552268
20170414-20170426,0.906977,0.784946,0.776699,0.787402,0.677419,0.643411,0.555556,0.653061,0.685950,0.663717,0.696411
20170508-20170520,0.857143,0.821782,0.841121,0.881481,0.905512,0.955224,0.947020,0.953642,0.920000,0.884956,0.906481
20170601-20170613,0.936170,0.849462,0.922330,0.953125,0.967480,0.976744,0.993103,1.000000,0.967213,0.946429,0.958534
20170625-20170707,0.914894,0.741935,0.776699,0.765625,0.756098,0.720930,0.820690,0.768707,0.688525,0.625000,0.763742
20170719-20170731,0.904762,0.953488,0.826531,0.762712,0.711712,0.745455,0.772727,0.781250,0.745283,0.744444,0.787325


Percentage of interferograms passes the requirement (thresthod = 0.683):

In [15]:
np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs

0.8775510204081632